In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('/content/drive/MyDrive/ML-things/dataset/agree75Ds.csv',
                 encoding = "ISO-8859-1")
df.dropna()
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df.head(5)

,label,text
0,1,"According to Gran , the company has no plans t..."
1,2,With the new production plant the company woul...
2,2,"For the last quarter of 2010 , Componenta 's n..."
3,2,"In the third quarter of 2010 , net sales incre..."
4,2,Operating profit rose to EUR 13.1 mn from EUR ...


In [3]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=42)

model = BertForSequenceClassification.from_pretrained(
    'yiyanghkust/finbert-pretrain', num_labels=3)

tokenizer = BertTokenizer.from_pretrained(
    'yiyanghkust/finbert-pretrain')

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [4]:
dataset_test

Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 346
})

In [5]:
def tokenize_data(data):
  return tokenizer(data['text'], truncation=True, padding="max_length",
                   max_length=128)

dataset_train = dataset_train.map(tokenize_data, batched=True)
dataset_val = dataset_val.map(tokenize_data, batched=True)
dataset_test = dataset_test.map(tokenize_data, batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids',
                                                'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids',
                                                'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids',
                                                'attention_mask', 'label'])

Map:   0%|          | 0/2796 [00:00<?, ? examples/s]

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

Map:   0%|          | 0/346 [00:00<?, ? examples/s]

In [8]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = predictions.argmax(-1)
  return {"accuracy": accuracy_score(predictions, labels)}

args = TrainingArguments(
    output_dir='temp/',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    report_to = 'none'
)

In [9]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.257906,0.909968
2,No log,0.187950,0.932476
3,No log,0.232077,0.916399


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=264, training_loss=0.23339197852394797, metrics={'train_runtime': 245.3295, 'train_samples_per_second': 34.191, 'train_steps_per_second': 1.076, 'total_flos': 551748836975616.0, 'train_loss': 0.23339197852394797, 'epoch': 3.0})

In [10]:
model.eval()
trainer.predict(dataset_test).metrics

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'test_loss': 0.2323138266801834,
 'test_accuracy': 0.9219653179190751,
 'test_runtime': 2.2835,
 'test_samples_per_second': 151.522,
 'test_steps_per_second': 4.817}

In [11]:
trainer.save_model('finbert-finetune/')

In [12]:
!zip -r finbert-finetune.zip finbert-finetune/

  adding: finbert-finetune/ (stored 0%)
  adding: finbert-finetune/training_args.bin (deflated 51%)
  adding: finbert-finetune/config.json (deflated 52%)
  adding: finbert-finetune/model.safetensors (deflated 7%)
